# Training od single CNN model

In [1]:
from siamese.dataset import DataLoader
from siamese.dataset import Dataset
from extensies import augmentation as a
from siamese.model import *
import tensorflow as tf
import numpy as np


### Data from mnist dataset

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_test /= 255

x_train = x_train.reshape((60000,28,28,1))
x_test = x_test.reshape((10000,28,28,1))
print('x_train shape:', x_train.shape)
print('Number of images in x_train', x_train.shape[0])
print('Number of images in x_test', x_test.shape[0])


x_train shape: (60000, 28, 28, 1)
Number of images in x_train 60000
Number of images in x_test 10000


### Data  from our PROSTATEx datast

In [3]:
modalities = ['adc/t/30x30x1']
if len(modalities) > 1:
    loader = DataLoader('../../data/',modalities)
    loader.load_data()
    loader.combine_channels(modalities)
    x_train, x_test, y_train, y_test = loader.get_train_test('combined')

else:
    loader = DataLoader('../../data/',modalities)
    loader.load_data()
    x_train, x_test, y_train, y_test = loader.get_train_test(modalities[0])
    
dataset = Dataset()
dataset.images_train = x_train
dataset.images_test = x_test
dataset.labels_train = y_train
dataset.labels_test = y_test

In [23]:
x,y = dataset.get_batch(5)

In [24]:
x.shape

(5, 30, 30, 1)

### Setting of model and optimalization

In [28]:
tf.reset_default_graph()

model = xmas_model
placeholder_shape = [None] + list(x_train.shape[1:])
x = tf.placeholder(tf.float32, placeholder_shape, name='x')
y = tf.placeholder(dtype = tf.int64, shape = [None], name = 'y')

net_output = model(x, reuse=False)

loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits = net_output, name = 'softmax'),name = 'loss')
 

correct_pred = tf.argmax(net_output, 1)
acc = tf.reduce_mean(tf.cast(tf.equal(correct_pred,y),tf.float32))


global_step = tf.Variable(0, trainable=False)
train_step = tf.train.AdamOptimizer(0.00001).minimize(loss, global_step = global_step)

print("logits: ", net_output)
print("loss: ", loss)
print("predicted_labels: ", acc)


logits:  Tensor("model/fc3/fc3/Relu:0", shape=(?, 2), dtype=float32)
loss:  Tensor("loss:0", shape=(), dtype=float32)
predicted_labels:  Tensor("Mean:0", shape=(), dtype=float32)


### Training of model

In [30]:
batch_size = 126
aug = False
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    #setup tensorboard	
    tf.summary.scalar('step', global_step)
    tf.summary.scalar('loss', loss)
    for var in tf.trainable_variables():
        tf.summary.histogram(var.op.name, var)
    merged = tf.summary.merge_all()

    #train iter
    for i in range(540):

        batch_x, batch_y = dataset.get_batch(batch_size,aug)
        # Run optimization op (backprop)x.
            # Calculate batch loss and accuracy
        loss_a,opt,acc_a = sess.run([loss, train_step, acc], feed_dict={x: batch_x, y: batch_y})
        
        
        if i % 20 == 0:
            loss_b,acc_b = sess.run([loss,acc], feed_dict={x: x_test, y: y_test})
        
            print("Iter " + str(i) + ", Loss= " + \
                      "{:.6f}".format(loss_a) + ", Loss test= {:.6f}".format(loss_b)+ ", Acc= {:.6f}".format(acc_b)+ ", AccA= {:.6f}".format(acc_a))
            
    pred = sess.run(correct_pred, feed_dict={x: x_test})
    print(pred)
    print_scores(y_test,pred)
            


Iter 0, Loss= 0.693139, Loss test= 0.693147, Acc= 0.752577, AccA= 0.746032
Iter 20, Loss= 0.684122, Loss test= 0.679791, Acc= 0.752577, AccA= 0.698413
Iter 40, Loss= 0.660988, Loss test= 0.660027, Acc= 0.752577, AccA= 0.777778
Iter 60, Loss= 0.622020, Loss test= 0.630170, Acc= 0.752577, AccA= 0.801587
Iter 80, Loss= 0.589200, Loss test= 0.590325, Acc= 0.752577, AccA= 0.753968
Iter 100, Loss= 0.560557, Loss test= 0.554637, Acc= 0.752577, AccA= 0.738095
Iter 120, Loss= 0.505978, Loss test= 0.541736, Acc= 0.752577, AccA= 0.785714
Iter 140, Loss= 0.592188, Loss test= 0.542355, Acc= 0.752577, AccA= 0.722222
Iter 160, Loss= 0.545471, Loss test= 0.541158, Acc= 0.752577, AccA= 0.753968
Iter 180, Loss= 0.496927, Loss test= 0.540481, Acc= 0.752577, AccA= 0.793651
Iter 200, Loss= 0.567344, Loss test= 0.540235, Acc= 0.752577, AccA= 0.722222
Iter 220, Loss= 0.542068, Loss test= 0.540924, Acc= 0.752577, AccA= 0.753968
Iter 240, Loss= 0.516450, Loss test= 0.539155, Acc= 0.752577, AccA= 0.777778
Iter 

In [9]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
def print_scores(labels, predicted_labels):
    print("Accuracy: {}".format(accuracy_score(labels, predicted_labels)))
    print("Precision: {}".format(precision_score(labels, predicted_labels)))
    print("Recall: {}".format(recall_score(labels, predicted_labels)))
    print("F1: {}".format(f1_score(labels, predicted_labels)))